# Processing Big Data - Deequ Analysis

© Explore Data Science Academy

## Honour Code
I {**YOUR NAME**, **YOUR SURNAME**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.


## Context

Having completed manual data quality checks, it should be obvious that the process can become quite cumbersome. As the Data Engineer in the team, you have researched some tools that could potentially save the team from having to do this cumbersome work. In your research, you have come a across a tool called [Deequ](https://github.com/awslabs/deequ), which is a library for measuring the data quality of large datasets.

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://github.com/Explore-AI/Pictures/raw/master/data_engineering/transform/predict/DataQuality.jpg"
     alt="Data Quality"
     style="float: center; padding-bottom=0.5em"
     width=100%/>
     <p><em>Figure 1. Six dimensions of data quality</em></p>
</div>

You present this tool to your manager; he is quite impressed and gives you the go-ahead to use this in your implementation. You are now required to perform some data quality tests using this automated data testing tool.
 

> ## 🚩️ Important Notice 🚩️
>
>To successfully run `pydeequ` without any errors, please make sure that you have an environment that is running pyspark version 3.0.
> You are advised to **create a new conda environment** and install this specific version of pyspark to avoid any technical issues:
>
> `pip install pyspark==3.0`

<br>

## Import dependencies

If you do not have `pydeequ` already installed, install it using the following command:
- `pip install pydeequ`

In [ ]:
""" 
Colab is essentially running on a linux machine on Google Cloud Platform.
This means that should you want to install something in your notebook you
would have to run headless installs as well as wget. Copying the installs
below will ensure that you have spark and java installed in the environment
as well as available for the notebook.
"""
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
"""
This section helps in rendering your notebook operable and ensuring that 
your environment variables for spark are correct. Running this cell in any
notebook allows for any unresolved spark environment to be fixed.
"""
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
os.environ["SPARK_VERSION"] = "3.0.0"

In [ ]:
"""
We need to locate Spark in the system. For that, we import findspark and 
use the findspark.init() method. If you want to know the location where 
Spark is installed, use findspark.find()
"""
import findspark
findspark.init()
# findspark.find()
import pyspark

In [ ]:
!pip install pydeequ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydeequ
from pydeequ.analyzers import *
from pydeequ.profiles import *
from pydeequ.suggestions import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType, DoubleType, IntegerType, DateType, NumericType, StructType, StringType, StructField

In [ ]:
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read data into spark dataframe

In this notebook, we set out to run some data quality tests, with the possiblity of running end to end on the years 1963, 1974, 1985, 1996, 2007, and 2018. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Data_ingestion_student_version.ipynb` notebook to create the parquet files for the following years:
>       - 1963
>       - 1974
>       - 1985
>       - 1996
>       - 2007
>       - 2018
>
>2. Ingest the data for the for the years given above. You should only do it one year at a time.
>3. Ingest the metadata file.


When developing your code, it will be sufficient to focus on a single year. However, after your development is done, you will need to run this notebook for all of the given years above so that you can answer all the questions given in the Data Testing MCQ.

In [ ]:
#TODO: Write your code here
# Use this variable (year) to determine which year your are focusing on
year = 1963
df = spark.read.parquet(f'/content/drive/MyDrive/data/Part_III/{year}')

In [ ]:
metadata_df = spark.read.csv('/content/drive/MyDrive/data/symbols_valid_meta.csv', header=True)

## **Run tests on the dataset**

## Test 1 - Null values ⛔️
For the first test, you are required to check the data for completeness.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for missing values in the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*


In [ ]:
#TODO: Write your code here
comp_test = VerificationSuite(spark) \
  .onData(df) \
  .addCheck(Check(spark_session = spark, level=CheckLevel.Error, description= 'Completeness checks') \
            .isComplete("date") \
            .isComplete("open") \
            .isComplete("high") \
            .isComplete("low") \
            .isComplete("close") \
            .isComplete("adj_close") \
            .isComplete("volume") \
            .areComplete(df.columns)) \
            .run()                                         

res = VerificationResult.checkResultsAsDataFrame(spark, comp_test)
res.show()

+-------------------+-----------+------------+--------------------+-----------------+------------------+
|              check|check_level|check_status|          constraint|constraint_status|constraint_message|
+-------------------+-----------+------------+--------------------+-----------------+------------------+
|Completeness checks|      Error|     Success|CompletenessConst...|          Success|                  |
|Completeness checks|      Error|     Success|CompletenessConst...|          Success|                  |
|Completeness checks|      Error|     Success|CompletenessConst...|          Success|                  |
|Completeness checks|      Error|     Success|CompletenessConst...|          Success|                  |
|Completeness checks|      Error|     Success|CompletenessConst...|          Success|                  |
|Completeness checks|      Error|     Success|CompletenessConst...|          Success|                  |
|Completeness checks|      Error|     Success|Completen

## Test 2 - Zero Values 🅾️

For the second test, you are required to check for zero values within the dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check for zero values within the data. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [ ]:
#TODO: Write your code here
zero_test = VerificationSuite(spark) \
  .onData(df) \
  .addCheck(Check(spark_session = spark, level=CheckLevel.Error, description= 'zero checks') \
            .isPositive("open") \
            .isPositive("high") \
            .isPositive("low") \
            .isPositive("close") \
            .isPositive("adj_close") \
            .isPositive("volume")) \
            .run()                                         

res = VerificationResult.checkResultsAsDataFrame(spark, zero_test)
res.show()

+-------------------+-----------+------------+--------------------+-----------------+--------------------+
|              check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+-------------------+-----------+------------+--------------------+-----------------+--------------------+
|Completeness checks|      Error|       Error|ComplianceConstra...|          Failure|Value: 0.45019920...|
|Completeness checks|      Error|       Error|ComplianceConstra...|          Success|                    |
|Completeness checks|      Error|       Error|ComplianceConstra...|          Success|                    |
|Completeness checks|      Error|       Error|ComplianceConstra...|          Success|                    |
|Completeness checks|      Error|       Error|ComplianceConstra...|          Success|                    |
|Completeness checks|      Error|       Error|ComplianceConstra...|          Failure|Value: 0.99940239...|
+-------------------+-----------+----

## Test 3 - Negative values ➖️
The third test requires you to check that all values in the data are positive.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to check negative values within the dataset. 
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [ ]:
#TODO: Write your code here
neg_test = VerificationSuite(spark) \
  .onData(df) \
  .addCheck(Check(spark_session = spark, level=CheckLevel.Error, description= 'negative checks') \
            .isNonNegative("date") \
            .isNonNegative("open") \
            .isNonNegative("high") \
            .isNonNegative("low") \
            .isNonNegative("close") \
            .isNonNegative("adj_close") \
            .isNonNegative("volume")) \
            .run()                                         

res = VerificationResult.checkResultsAsDataFrame(spark, neg_test)
res.show()

+---------------+-----------+------------+--------------------+-----------------+------------------+
|          check|check_level|check_status|          constraint|constraint_status|constraint_message|
+---------------+-----------+------------+--------------------+-----------------+------------------+
|negative checks|      Error|     Success|ComplianceConstra...|          Success|                  |
|negative checks|      Error|     Success|ComplianceConstra...|          Success|                  |
|negative checks|      Error|     Success|ComplianceConstra...|          Success|                  |
|negative checks|      Error|     Success|ComplianceConstra...|          Success|                  |
|negative checks|      Error|     Success|ComplianceConstra...|          Success|                  |
|negative checks|      Error|     Success|ComplianceConstra...|          Success|                  |
|negative checks|      Error|     Success|ComplianceConstra...|          Success|          

## Test 4 - Determine Maximum Values ⚠️

For the fourth test, we want to find the maximum values in the dataset for the numerical fields. Extremum values can often be used to define an upper bound for the column values so we can define them as the threshold values. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Column Profiler Runner` to generate summary statistics for all the available columns. 
>2. Extract the maximum values for all the numeric columns in the data.
>
> *You may use as many cells as necessary*

In [ ]:
#TODO: Write your code here
num_cols = ["open", "low", "close", "high", "adj_close", "volume"]
result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
  if col in num_cols:
    print(f'Maximum value for {col} is {profile.maximum}')
    

Maximum value for open is 303.125
Maximum value for low is 311.875
Maximum value for close is 313.75
Maximum value for volume is 20692800.0
Maximum value for adj_close is 148.7704620361328
Maximum value for high is 315.625


## Test 5 - Stock Tickers 💹️

For the fifth test, we want to determine if the stock tickers contained in our dataset are consistent. To do this, you will need to make use of use of the metadata file to check that the stock names used in the dataframe are valid. 

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine if the stock tickers contained in the dataset appear in the metadata file.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*

In [ ]:
metadata_df = metadata_df.toPandas()
metadata_symbol = [i for i in metadata_df.Symbol]

In [ ]:
#TODO: Write your code here

consistency_test = VerificationSuite(spark) \
  .onData(df) \
  .addCheck(Check(spark_session = spark, level=CheckLevel.Error, description= 'consistency checks') \
            .isContainedIn("stock",metadata_symbol)) \
            .run()                                         

res = VerificationResult.checkResultsAsDataFrame(spark, consistency_test)
res.show()

+---------------+-----------+------------+--------------------+-----------------+------------------+
|          check|check_level|check_status|          constraint|constraint_status|constraint_message|
+---------------+-----------+------------+--------------------+-----------------+------------------+
|negative checks|      Error|     Success|ComplianceConstra...|          Success|                  |
+---------------+-----------+------------+--------------------+-----------------+------------------+



## Test 6 - Duplication 👥️
Lastly, we want to determine the uniqueness of the items found in the dataframe. You need to make use of the Verification Suite to check for the validity of the stock tickers. 

Similar to the previous notebook - `Data_profiling_student_version.ipynb`, the first thing to check will be if the primary key values within the dataset are unique - in our case, that will be a combination of the stock name and the date. Secondly, we want to check if the entries are all unique, which is done by checking for duplicates across that whole dataset.

> ℹ️ **Instructions** ℹ️
>
>1. Make use of the `Verification Suite` and write code to determine the uniqueness of entries contained within the dataset.
>2. Display the results of your test.
>
> *You may use as many cells as necessary*



In [ ]:
#TODO: Write your code here
uni_test = VerificationSuite(spark) \
  .onData(df) \
  .addCheck(Check(spark_session = spark, level=CheckLevel.Error, description= 'uniqueness checks') \
            .hasUniqueness(("date","stock"), lambda x: x == 1)) \
            .run()                                         

res = VerificationResult.checkResultsAsDataFrame(spark, uni_test)
res.show()

Python Callback server started!
+-----------------+-----------+------------+--------------------+-----------------+------------------+
|            check|check_level|check_status|          constraint|constraint_status|constraint_message|
+-----------------+-----------+------------+--------------------+-----------------+------------------+
|uniqueness checks|      Error|     Success|UniquenessConstra...|          Success|                  |
+-----------------+-----------+------------+--------------------+-----------------+------------------+

